In [1]:
import mediapipe as mp
import cv2
import numpy as np
import os 

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

def determine_winner(hand_labels):
    count = {"Rock": 0, "Paper": 0, "Scissors": 0, "Unknown": 0}
    for label in hand_labels:
        if label[0] in count:
            count[label[0]] += 1


    # Determine the winner based on the counts
    if count["Unknown"] > 0:
        return None
    elif count["Rock"] > 0 and count["Paper"] == 0 and count["Scissors"] == 0:
        return "Draw"
    elif count["Rock"] == 0 and count["Paper"] > 0 and count["Scissors"] == 0:
        return "Draw"
    elif count["Rock"] == 0 and count["Paper"] == 0 and count["Scissors"] > 0:
        return "Draw"     
    elif count["Rock"] > 0 and count["Paper"] > 0 and count["Scissors"] > 0:
        return "Draw"
    elif count["Rock"] > 0 and count["Paper"] > 0:
        return "Paper"
    elif count["Rock"] > 0 and count["Scissors"] > 0:
        return "Rock"
    elif count["Paper"] > 0 and count["Scissors"] > 0:
        return "Scissors"
    else:
        return None





def condition_rock(ip, it, mp, mt, rp, rt, pp, pt):
    if (ip < it) and (mp < mt) and (rp < rt) and (pp < pt):
        return True
    return False

def condition_paper(ip, it, mp, mt, rp, rt, pp, pt):
    if (ip > it) and (mp > mt) and (rp > rt) and (pp > pt):
        return True
    return False

def condition_scissors(ip, it, mp, mt, rp, rt, pp, pt):
    if (ip > it) and (mp > mt) and (rp < rt) and (pp < pt):
        return True
    return False

def determine_gesture(hand_landmarks, hand_side):
    ip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y
    it = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    mp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y
    mt = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
    rp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_PIP].y
    rt = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y
    pp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_PIP].y
    pt = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y

    if condition_rock(ip, it, mp, mt, rp, rt, pp, pt):
        return "Rock", hand_side
    elif condition_paper(ip, it, mp, mt, rp, rt, pp, pt):
        return "Paper", hand_side
    elif condition_scissors(ip, it, mp, mt, rp, rt, pp, pt):
        return "Scissors", hand_side
    else:
        return "Unknown", hand_side


cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=6) as hands:
    while cap.isOpened():
        ret, frame = cap.read()

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Flip on horizontal
        image = cv2.flip(image, 1)

        # Set flag
        image.flags.writeable = False

        # Detections
        results = hands.process(image)

        # Set flag to true
        image.flags.writeable = True

        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        hand_labels = []
        if results.multi_hand_landmarks:
            for num, hand_landmarks in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                          mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2))

                hand_side = "Left" if hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x < 0.5 else "Right"
                gesture_label = determine_gesture(hand_landmarks,hand_side)
                hand_labels.append(gesture_label)

                text = 'Hand {}: {}'.format(num+1, gesture_label)
                coords = tuple(np.multiply(
                    np.array((hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x, hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y)),
                    [640, 480]).astype(int))
                cv2.putText(image, text, coords, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                # Determine text position
                text_position = (int(hand_landmarks.landmark[0].x * image.shape[1]), int(hand_landmarks.landmark[0].y * image.shape[0]) + 20)

                # Add text under hand
                cv2.putText(image, f"Hand {num+1}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
        winner = determine_winner(hand_labels)
        if winner:
            cv2.putText(image, "Winner: " + winner, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                



        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
